In [31]:
import os
import pandas as pd
import openpyxl
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


In [55]:
nombre_archivo = ["Presupuesto_mensual_2021.xlsx", "Presupuesto_mensual_2022.xlsx", "Presupuesto_mensual_2023.xlsx", "Presupuesto_mensual_2024.xlsx"]
rango_hojas = list(range(1,13))

def excel_a_df(archivo, rango_hojas):
    df_compras = pd.DataFrame() #df vacio para almacenar los resultados
    for a in archivo:
        try:
            datos = pd.read_excel(a, sheet_name= rango_hojas, header= None, skiprows= 5)
            #concatenar todas las hojas de los arhicvos en el df final
            for hoja, df in datos.items():
                df_compras = pd.concat([df_compras, df], ignore_index= True) #directamente concatena
            #manejo de errores
        except ValueError as e:
            #print(f"Algunas hojas especificadas no existen en {archivo}: {e}")
            continue
        except Exception as e:
            #print(f"Error inesperado al procesar el libro {a} y la hoja {l}")
            continue
    return df_compras 


In [56]:
df = excel_a_df(nombre_archivo, rango_hojas)

In [57]:
#pd.set_option('display.max_rows', None)  # Mostrar todas las filas
df.head()

,0,1,2,3,4,5,6,7,8,9
0,N,2021-12-30,3.2,Perforadora,NaN,Chino,Vivienda,NaN,-44560.0,NaN
1,N,2021-12-30,1.2,Pincel de silicona,NaN,Chino,Vivienda,NaN,-44560.0,NaN
2,N,2021-12-30,0.5,Regla,NaN,Chino,Vivienda,NaN,-44560.0,NaN
3,N,2021-12-30,2.5,Resaltadores,NaN,Chino,Vivienda,NaN,-44560.0,NaN
4,N,2021-12-30,2.0,Cuaderno,NaN,Chino,Vivienda,NaN,-44560.0,NaN


In [58]:
df_completo = df.drop([0,4,7,8,9], axis = 1)
#verificar que se han eliminado las columnas

In [ ]:
df_completo


In [70]:
df_completo.columns = ['Fecha', 'Precio', 'Producto', 'Lugar', 'Categoria']
df_completo.sample()

,Fecha,Precio,Producto,Lugar,Categoria
1495,2023-01-07,1.0,caramelos miel,NaN,Comidas


In [74]:
df_completo.dtypes

Fecha        datetime64[ns]
Precio              float64
Producto             object
Lugar                object
Categoria            object
dtype: object

In [75]:
# Convertir a datetime y eliminar la hora
df_completo['Fecha'] = pd.to_datetime(df_completo['Fecha'])
print(df_completo['Fecha'].dtype)  # Esto será: datetime64[ns]

datetime64[ns]


In [69]:
df_completo['Precio'] = pd.to_numeric(df_completo['Precio'], errors='coerce')
df_completo['Precio'].dtype

dtype('float64')